In [1]:
import numpy as np
import time

### Conventional DP Approach 

In [4]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_global(v, w, pointers):
    i,j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])



def global_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as
    computed by traceback_global.

    :param: v
    :param: w
    :param: delta
    """
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
    score, alignment = None, None
    for i in range(len(v)+1):
      for j in range(len(w)+1):
        if i == 0 and j == 0:
          continue
        elif i == 0:
          M[i][j] = M[i][j - 1] + delta["-"][w[j-1]]
          pointers[i][j] = LEFT
        elif j == 0:
          M[i][j] = M[i - 1][j] + delta[v[i-1]]["-"]
          pointers[i][j] = UP
        else:
          min_num = min(M[i-1][j] + delta[v[i-1]]["-"], M[i][j-1] + delta["-"][w[j-1]], M[i-1][j-1] + delta[v[i-1]][w[j-1]])
          if min_num == M[i-1][j-1] + delta[v[i-1]][w[j-1]]:
            M[i][j] = min_num
            pointers[i][j] = TOPLEFT
          elif min_num == M[i][j-1] + delta["-"][w[j-1]]:
            M[i][j] = min_num
            pointers[i][j] = LEFT
          elif min_num == M[i-1][j] + delta[v[i-1]]["-"]:
            M[i][j] = min_num
            pointers[i][j] = UP
    # for row in M:
    #   print(row)
    alignment = traceback_global(v,w, pointers)
    return M[-1][-1], alignment


keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [0 if keys[i] == keys[j]  else 1 for j in range(len(keys))])}

# score, alignment = global_align("TATGC", "AGTACGCA", delta)
# score, alignment = global_align("CACACC", "ACCACCCAA", delta)
score, alignment = global_align("CACCAA", "ACCACC", delta)
# # score, alignment = global_align("CAC", "ACC", delta) #ROW, COL
# print(score, alignment, sep='\n')
# print("Score: ", score)

### Helper Functions for FOUR RUSSIANS

In [5]:
def getAllStrings(t, alphabets):
    if t == 0:
        return []
    if t == 1:
        return alphabets
    tmp = getAllStrings(t-1, alphabets)
    res = []
    for i in alphabets:
        for j in tmp:
            res.append(i+j)
    return res

# create random strngs of length x from chracters in alphabets
def createRandomString(x, alphabets):
    res = ''
    for i in range(x):
        res += alphabets[np.random.randint(0, len(alphabets))]
    return res

# get all possible vectors of length t consisting of 1, 0, -1
def getAllVectors(t, lst):
    if t == 0:
        return [[]]
    tmp = getAllVectors(t-1, lst)
    res = []
    for i in lst:
        for j in tmp:
            res.append([i]+j)
    return res

def makeString(lst):
    res = ''
    for i in lst:
        res += str(i)
    return res

In [6]:
def getBlockRowCol(u, v, row1, col1, A=0):
    block = np.zeros((len(v)+1, len(u)+1))
    block[0, 0] = A
    block[0, 1:] = np.cumsum(row1)
    block[1:, 0] = np.cumsum(col1)
    for i in range(1, len(v) + 1):
        for j in range(1, len(u) + 1):
            if v[i-1] == u[j-1]:
                block[i][j] = block[i-1][j-1]
            else:
                block[i][j] = min(block[i-1][j-1], block[i-1][j], block[i][j-1]) + 1
    
    lastRow = np.diff(block[-1, :])
    lastCol = np.diff(block[:, -1])

    del block
    return lastRow, lastCol

def finalBlocksGetter(alphabets, t):
    # get all possible strings of length t
    allstrs = getAllStrings(t, alphabets)
    allnums = getAllVectors(t, [1, 0, -1])
    allnumsstr = [makeString(i) for i in allnums]
    lenallnums = len(allnums)

    allBlocks = {i: {j: None for j in allstrs} for i in allstrs}
    for i in allstrs:
        for j in allstrs:
            allBlocks[i][j] = {r2: {r: None for r in allnumsstr} for r2 in allnumsstr}

    for i in allstrs:
        for j in allstrs:
            for indexRow in range(lenallnums):
                r = allnums[indexRow]
                r_str = allnumsstr[indexRow]
                for indexCol in range(lenallnums):
                    c = allnums[indexCol]
                    c_str = allnumsstr[indexCol]
                    allBlocks[i][j][r_str][c_str] = getBlockRowCol(i, j, r, c)

    return allBlocks

## Four Russians Implementation

In [7]:
def FR(u, v, t, precomputedBlocks):
    u_pad = u
    v_pad = v
    block = np.zeros((len(v_pad)+1, len(u_pad)+1))
    block[0, :] = np.arange(len(u_pad) + 1)
    block[:, 0] = np.arange(len(v_pad) + 1)
    block[0, 0] = 0
    block_size = t
    
    for i in range(0, len(v_pad), block_size):
        for j in range(0, len(u_pad), block_size):
            row1 = np.diff(block[i, j:j+block_size+1])
            col_tmp = block[i:i+block_size+1, j].copy()
            col1 = np.diff(col_tmp)
            row_diff = np.int32(row1)
            col_diff = np.int32(col1)
            row_diff_str = makeString(row_diff)
            col_diff_str = makeString(col_diff)
            row_substring = u[j:j+block_size]
            col_substring = v[i:i+block_size]  
            
            r_s, t_s = precomputedBlocks[row_substring][col_substring][row_diff_str][col_diff_str]
            block[i+block_size, j+1:j+block_size+1] = r_s
            block[i+block_size, j+1:j+block_size+1] = np.cumsum(block[i+block_size, j:j+block_size+1])[1:]
            block[i+1:i+block_size+1, j+block_size] = t_s
            block[i+1:i+block_size+1, j+block_size] = np.cumsum(block[i:i+block_size+1, j+block_size])[1:]
    return block, block[-1, -1]

In [8]:
allBlocks = finalBlocksGetter(['A', 'C'], 3)

In [9]:
def FR2(u, v, t, precomputedBlocks):
    u_pad = u
    v_pad = v
    block = np.zeros((len(v_pad)+1, len(u_pad)+1))
    block[0, :] = np.ones(len(u_pad) + 1)
    block[:, 0] = np.ones(len(v_pad) + 1)
    block[0, 0] = 0
    block_size = t
    
    for i in range(0, len(v_pad), block_size):
        for j in range(0, len(u_pad), block_size):
            row1 = block[i, j+1:j+block_size+1]
            col1 = block[i+1:i+block_size+1, j].copy()
            
            row_diff = np.int32(row1)
            col_diff = np.int32(col1)
            row_diff_str = makeString(row_diff)
            col_diff_str = makeString(col_diff)
            row_substring = u[j:j+block_size]
            col_substring = v[i:i+block_size]  
            print("Input", row_substring, col_substring, row_diff_str, col_diff_str)
            r_s, t_s = precomputedBlocks[row_substring][col_substring][row_diff_str][col_diff_str]
            print("Output", r_s, t_s)
            block[i+block_size, j+1:j+block_size+1] = r_s
            # block[i+block_size, j+1:j+block_size+1] = np.cumsum(block[i+block_size, j:j+block_size+1])[1:]
            block[i+1:i+block_size+1, j+block_size] = t_s
            # block[i+1:i+block_size+1, j+block_size] = np.cumsum(block[i:i+block_size+1, j+block_size])[1:]

    first_col_sum = np.sum(block[:, 0])
    first_row_sum = np.sum(block[0, :])
    last_row_sum = np.sum(block[-1, :])
    last_col_sum = np.sum(block[:, -1])
    print(first_row_sum, first_col_sum, last_row_sum, last_col_sum)
    sc1 = first_row_sum + last_col_sum - block[0, -1]
    sc2 = first_col_sum + last_row_sum - block[-1, 0]
    print("SCORES", sc1, sc2)
    return block, block[-1, -1]

GroundTruthBlock, sc = FR2("ACCACC", "CACCAA", 3, allBlocks)
# GroundTruthBlock = FR("ACCACCCAA", "CACACC", 3, allBlocks)
print("FINISHED")
print(GroundTruthBlock)
print(sc)

Input ACC CAC 111 111
Output [-1. -1.  1.] [-1.  0.  0.]
Input ACC CAC 111 -100
Output [ 1. -1.  1.] [-1. -1. -1.]
Input ACC CAA -1-10 111
Output [-1. -1. -1.] [0. 1. 1.]
Input ACC CAA 1-1-1 011
Output [-1.  0.  1.] [1. 1. 1.]
6.0 6.0 0.0 1.0
SCORES 6.0 5.0
FINISHED
[[ 0.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0. -1.  0.  0. -1.]
 [ 1.  0.  0.  0.  0.  0. -1.]
 [ 1. -1. -1.  0.  1. -1. -1.]
 [ 1.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  1.]
 [ 1. -1. -1.  1. -1.  0.  1.]]
1.0


In [10]:
import numpy as np
import concurrent.futures
import time


def process_block(i, j, block, block_size, precomputedBlocks, u, v):
    row1 = np.diff(block[i, j:j + block_size + 1])
    col_tmp = block[i:i + block_size + 1, j].copy()
    col1 = np.diff(col_tmp)
    row_diff = np.int32(row1)
    col_diff = np.int32(col1)
    row_diff_str = makeString(row_diff)
    col_diff_str = makeString(col_diff)
    row_substring = u[j:j + block_size]
    col_substring = v[i:i + block_size]
    r_s, t_s = precomputedBlocks[row_substring][col_substring][row_diff_str][col_diff_str]
    block[i + block_size, j + 1:j + block_size + 1] = r_s
    block[i + block_size, j + 1:j + block_size + 1] = np.cumsum(block[i + block_size, j:j + block_size + 1])[1:]
    block[i + 1:i + block_size + 1, j + block_size] = t_s
    block[i + 1:i + block_size + 1, j + block_size] = np.cumsum(block[i:i + block_size + 1, j + block_size])[1:]
        
def FR_Parallel(u, v, t, precomputedBlocks, delay=0.0001):
    u_pad = u
    v_pad = v
    block = np.zeros((len(v_pad) + 1, len(u_pad) + 1))
    block[0, :] = np.arange(len(u_pad) + 1)
    block[:, 0] = np.arange(len(v_pad) + 1)
    block[0, 0] = 0
    block_size = t

    def thread_worker(i, u_pad, v_pad):
        for j in range(0, len(u_pad), block_size):
            process_block(i, j, block, block_size, precomputedBlocks, u_pad, v_pad)

    with concurrent.futures.ThreadPoolExecutor() as outer_executor:
        for i in range(0, len(v_pad), block_size):
            outer_executor.submit(thread_worker, i, u_pad, v_pad)
            time.sleep(delay)  # Add a small delay before starting the next outer thread because we need a block already done to be able to start the next one

    return block, block[-1, -1]

# GroundTruthBlock, sc1 = FR_Parallel("ACCACC", "CACCAA", 3, allBlocks)
# # GroundTruthBlock = FR("ACCACCCAA", "CACACC", 3, allBlocks)
# print("FINISHED")
# print(GroundTruthBlock)
# print(sc1)

In [9]:
t = 3
alphabets = ['A', 'C']
start_time = time.time()
allBlocks = finalBlocksGetter(alphabets, t)
end_time = time.time()
time_taken_all_blocks = end_time - start_time
print("Block size", t)
print("ALL BLOCKS - PRECOMPUTE TIME", time_taken_all_blocks)

Block size 3
ALL BLOCKS - PRECOMPUTE TIME 2.1911110877990723


In [10]:
z = t*500
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  1500
STR1: CAACCCAACCACAAAAAACCCACAACACCCAACCCCCACAACAAAACAACAAAACACAAAACCACCACAAACCAACAAACAAACCCCAAACCCCCACACACAACCAACCCAACCCCACACCCCACCACAAACAACAAAAACCACCACCCCCAAACAAACCACACAACAAACACAAACCACCAAAACACCCACCAAAAACCACACCAAAAAAAACAAAACCCACAAAACACACAACAAACCCACCACACACCCCCCCCCAACCACACCCCCACAAACCCACCCCAAACAACCAACCCAAAACAACAACCCACACACCCACCCAAAAACCACACCCACAACACCAACCACAACCAACCCAACCAACAACCCAACCCAAACAAAACCAACCCAAACACCACACCCAAACCACCACAAACAACCCACCCAACAACACAACAAAAAAAAACCCCCAACCCCCCAAACCAAAAAACACACACCCCCCAAAACCAAACACCCCCCCACCCACACCAAACCAACACAACCACCCAAACAAACACCAACCAACACACAAACAACCCAACCAACCAACCAAACAAAAACAACAAAAACCACCACCACACACACCCACCAAAAACAAAAAAACACCACCCAACAACCACAACAAACCACCCACCCACCCAAAACACCAAACCAACCCCACCCACACAACAACCCCCAACACAAACCCACCCCCAACCAAACCCCACACCCCCACAACACAACAACACACACAACCAAACCCCACCAAAAAACACACCCAAACCCACCCACACCCCCAAACAACCCACAACCAAAACCCCACCCCCCCCACAAAAACAACAAACACAAAACAACAACAACCCCACAAACCAAACAACAACAAACCACCACCAACAAACCAACACCCAAAACCACACAAAACAACCCCCAACACCACCCCACCCCCACCACACACACACACCAACCCAACAA

### Testing for string length 24 from the following code block (t=3), alphabets A, C, G,T

In [26]:
t = 3
alphabets = ['A', 'C', 'T', 'G']
start_time = time.time()
allBlocks = finalBlocksGetter(alphabets, t)
end_time = time.time()
time_taken_all_blocks = end_time - start_time
print("Block size", t)
print("ALL BLOCKS - PRECOMPUTE TIME", time_taken_all_blocks, "seconds.")

Block size 3
ALL BLOCKS - PRECOMPUTE TIME 76.00015187263489 seconds.


In [27]:
z = 24
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.05)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  24
STR1: CTCGAAAACGTTATGAGGCTACTG
STR2: ATGGCGGCAATCTCTCAATCAAAC
DP EDIT SCORE: 16    FR SCORE: 16.0    PARALLEL FR SCORE: 16.0
DP EDIT TIME: 0.0009996891021728516    FR TIME: 0.008780241012573242    PARALLEL FR TIME: 0.4302492141723633


### Testing for string length 48 from the following code block (t=3) alphabets A, C, G,T

In [28]:
z = 48
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.1)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  48
STR1: CAGCACTGTCACGACTCATAGGTGGCAGAATTGCGGTACATAACACCT
STR2: CGTCAGATCGTTTCGATAAAGCCTTTAATATCTGAATCGTCCACGACG
DP EDIT SCORE: 28    FR SCORE: 28.0    PARALLEL FR SCORE: 28.0
DP EDIT TIME: 0.0014772415161132812    FR TIME: 0.007238626480102539    PARALLEL FR TIME: 1.6685574054718018


### Testing for string length 132 from the following code block (t=3) alphabets A, C, G,T

In [29]:
z = 132
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  132
STR1: TTCGTGGATTGCAGCGAGATTTCGAGTTACGAATGTCCTAAGGGTCTTTTCTTCTTGACTAGAGGGGGTAGCACCCTCGCTCTACCTATTTGCACGGGGAACAGGAGGCCCATTTGTTTTTTTTCTCCGTCG
STR2: ACTAGGCCACGCTGTTCTTCATAAGGATATCGTTATTTCTCGGAACGATTAGGATGTATGGGCTGTTCATTGCCTGGCCTACCAGAGAATGCCTGAGAACAAAGAAAACATCGACGAGGATGTACCACATTT
DP EDIT SCORE: 75    FR SCORE: 75.0    PARALLEL FR SCORE: 75.0
DP EDIT TIME: 0.014417409896850586    FR TIME: 0.035115718841552734    PARALLEL FR TIME: 0.588878870010376


### Testing for string length 516 from the following code block (t=3) alphabets A, C, G,T

In [31]:
z = 516
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  516
STR1: TTGAACATGAAACTTATCACATAGTACGTGATAGAGCGTATCAATCTTATGAGGGTTCGGACGCAATAGGGCCGCGTATGTTTTCCTAACGTCGCAGTGTTTGGCACCAGGAAAATGTAACGGGTATTATGGCATCAGAAATTGTGGCGACCGCCACAACCCAAATGGGGTACTTCCGTCAAGACCCGCTTATTAGGCAGAGGTTCAATATCACGATGGGGTCCGGGCTGGTGGGGTGAACCCACCGTATTGGAAATCGTGATGCGCTGTTGCGTGGTGGACACAAACCGTTGTTGTCAGCTGTCAAACATGTTGTACAATTTTGAAATACTACTGAAGGTACTGTGACCTTTTGTTGTACTAATTGGTGAACAGGGCCTCGCTGGCTATATCACCTGCCCTACGGGTTAAATCGACGGATTATCTACTTGACTTTTCTTAGTCAAAAGTACTTGAATGACTATTGAGAAGTATGACTCGCAACCTTTTAGCCAAATCAAAAAATGCCGATGCCGG
STR2: CTGACAAACCTGGTTAGCACCTTTCTCTGCTTAGTGGGGAGTGACGTCACAGCGCGGCTACTCCCAGTTTAGGTGGGCCTCCACAACAGCCCTGTTTCCCTTCACCGCAAACGTCCCTCTCCAACATGTAGGTAGTGCACCAGACAGTTCAATAGGAGACGGTCGACGTGTGAAGGACAAGCTGATTGTAGGGAATGAAAAGTACCGTAGAAGCTTTCAGCGGCAGGTCATAAGAGGAGCGGTGCAGCTTACTGGACAGCAAGCTACCACAATCGCCCCGTACCCAAGCTACCGTATCACATGCAGCGGGTCACAAAGTGCCGGGTGAGTATGAGCAGACCACAAGGCAGCGTTGACTATCCCATCTGCCAACGGACCACGGCTAGAATTTTTAACAAAACATACCCATCCGTTAAGCCCGTTCCACTTGCGTAATGCCCGCTGACC

### Testing for string length 1188 from the following code block (t=3) alphabets A, C, G,T

In [32]:
z = 1188
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  1188
STR1: CATTCTCAACAGGACCTTTTCGCTGGTGGCAAGGAAGAAAAAATCCACCTTATTTGCACCACCAAAATATTGTTTTCCAGGCGCCTGGTTGTCCGATAACACGGCCTGGAATCAGCATGGAGGTGCGAAAGAGTGGACCAGCAGTGCATGGCAGTCGAGCCTGTCTGTCTGCGTTTAGGCCAGGGGTCAATAAGTTTAGCATAGTTTCCGAGGGCCGAGGCACAAATGATTATCGACTATAACCGTTTATGTGGCAGTCTCTGCGCTTTGCATAGACCTATTGGTGTAGCGGACTCCTGGACTCGTCGTTGGGGCACAATGTTCTTAAAACCTCTCGCTAAGCCTTCACGCTCTTCATTGCTTCTACACTCTATACCTAGGTTGTTCTAAACATGCTGACTTCAGGACGCAAAGTGTCTTTAACGGCGCTACGAATGCAAGGAAGCCAGTCTGCGGTGGTCAATCCACATGCCTCAGAGGGTCGGGCTAGATGCCCACACGAGATATCGTGAAACACGCTCGTTAGTAGCAGCCCCCTGTCCCCACCACGCCGGTAAACTGCTGCCCAGGCAACGTTCAACAACTGTCAAAGAACTAAGCCTTCTAGCGTACACTCCTGGAAGTACGCCCCAGAAATGTATTACCAGTGGTAGTGGCAACGTCGTTCTCTCTGTAATCACCACGCTCATCGTTCTCCAATGAGTGTTATTACAATCACCAACCCACGGGTAGTTCAATATCCCAGCGATGTACTCTCTACGGGACACGTCTCATAGAGTATAGCGCGCGATACTGCGGCTGGGATCTCCGGAAATGCATATGATCGTAACTTACCGCATCAAGACGTCATGATAACGTCACCATATGCCAGCGCGTCCGCTCCAGTATCGTTCTAATAGTCCCCCACGGCGGGTCCGCTTGCGGGCCGCATTTCTCATCTGGGTTGCTCGACATTTAAAACCCTCCTTC

### Testing for string length 2400 from the following code block (t=3) alphabets A, C, G,T

In [33]:
z = 2400
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  2400
STR1: TGCAAATAGTGGTGCTTAGTCATTCTAGAATCATATTCGGAGAATAAATCGACTAAATTCGTCCTAGACCACTAAGTATATATTACTACAGCTACGACATTCGGCGCCTTCCATTTTCTAAGTCCCTTCGTAAGAGCCTTGTCTATTTAGTTAATCCCTCTCTGTCCCTCGAATCCGCAAAGTTAATCCGACCGCGGCCGATAGCCATGGGGAACGTTTTAGGAGTTCTCGTCGCAACTATCTAAGCAAGCACACAACATGAGGCTCCGCGGTACAAGTTAGCTAGACCCTACACTTGTAGAGGTATGATTTAGGTCACTCGATTGTTATCCCACGCCGAAAAGTCTAATCATGTGTACGAGCCAAGCGGACGTCAGGTGACCTAGGCCAAGCTAGGGGGCCGTCTGCCGGTCCAACCATGAAGGAAGTCCCAGATTATTTAGCACCTGAGGGGCGTTACTTGCCCCTCCGGGTTCCCGAATCCAGATGGCACTTTCCAGCGGGCAGCACCAGGGTATTCGGCAAGCTCAAACTCACCTGCCCATTGGTTGGTCGAGTAGCCTTCTCCAGTGACTGCAGCGAGATCCCGCACAGCGGCATGTTGCACTGTACGTAGATTGTGTGCTTATCCCGAGATAGTGGTACAGTGGGGAAGTTCAAGTGACTAACTCAACCTAGCGAGTCCTTCGACGTTTATAGAACGCGCGACCTAACAACCCTGGCGCGTGATGAGAGGCGATTTCCAACTCGCTATCGGGATATCGACCTGTATTAGCTGTACCAGATAGGAAACCATGACTCACGGATAATGGGTGCTTGCTTGTGTTTGCAGCAAGTGCTCTTTCTGGCTCAGAACGGCCCACAAACACCGACGAAAAGGACAGACAAACGAACCCACGTCACCTAATTCAAACCTATTCCGTTTCCTCTAGTCTCTGAACACCTGCACTTACTCCCTTTCGAAGTTGA

############################################################################################################################
############################################################################################################################
############################################################################################################################

### Testing for string length 24 from the following code block (t=4) alphabets A, C

In [12]:
t = 4
alphabets = ['A', 'C',]
start_time = time.time()
allBlocks = finalBlocksGetter(alphabets, t)
end_time = time.time()
time_taken_all_blocks = end_time - start_time
print("Block size", t)
print("ALL BLOCKS - PRECOMPUTE TIME", time_taken_all_blocks, "seconds.")

Block size 4
ALL BLOCKS - PRECOMPUTE TIME 67.65768504142761 seconds.


In [13]:
z = 24
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.05)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  24
STR1: ACAACACCACAAACAAAACAAACC
STR2: ACAAACAACACAAACCACAACAAA
DP EDIT SCORE: 6    FR SCORE: 6.0    PARALLEL FR SCORE: 6.0
DP EDIT TIME: 0.0    FR TIME: 0.005674839019775391    PARALLEL FR TIME: 0.35131001472473145


### Testing for string length 48 from the following code block (t=4) alphabets A, C

In [14]:
z = 48
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.1)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  48
STR1: AACCCACCCCCAAACCCAACCACCAAAAACCACACACCCCCACCACAA
STR2: AAAACCCCACACCACCCCCAAAAAAACCAACCAAAACCAACACCAACA
DP EDIT SCORE: 18    FR SCORE: 18.0    PARALLEL FR SCORE: 18.0
DP EDIT TIME: 0.003608226776123047    FR TIME: 0.013235330581665039    PARALLEL FR TIME: 1.2668960094451904


### Testing for string length 132 from the following code block (t=4) alphabets A, C

In [15]:
z = 132
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  132
STR1: ACCCCACAAAAACCAACCCACCACCCCCACACACCACAAACCACCAACCCAAAAAAACAACACCAACCCAAACCCCACCACCCCCAACCCCAAACACCCCCAACCAAACACACCCAACAACACCAAACAAAA
STR2: CCACCACCAAAAAACCACCCCCCCAACAAACAACCCAACCCCCAACCACAAAAACCACACCACCCAACACAAACCCCCAACACCAACAACCAAACAAACCCAACACACACCCACCCACAAAACACACACCCC
DP EDIT SCORE: 38    FR SCORE: 38.0    PARALLEL FR SCORE: 38.0
DP EDIT TIME: 0.030030250549316406    FR TIME: 0.04444265365600586    PARALLEL FR TIME: 0.40308237075805664


### Testing for string length 516 from the following code block (t=4) alphabets A, C

In [16]:
z = 516
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  516
STR1: AAACACCACCCCCACCCAAAACAACAACAAAACCACACACACCACCCAACACCAACCACACCAACAAAACAACAACACCAAACAAAACAAAAACAAAACACCAACCCACCAACCACCCACCCCAACAACAAACAAAACAAACCACACACCAAAACACAACCCCAACAAACCCACCCACAACCCCCAAAAACAAAAACACCCCCAACACCACACAACACAACACACACCAACACCCCACAAACACACCCCACCCCCACACCAAAAAACCCCACACCCAAAACCACACACCCACCACACAACAACCCAAAAACACCAACCAAAAAACAACACCCACACACAAACCCACAACAACAAAACAACACAACCAACCCCAAAACACACCAACAACCAAAACACAACCCCCAAAAAAACACCCCACCAAACCACACAACAACCCAACCCACCACACAACCCACCCACCCCCACCACAAAAAACAAACCCCAACAAAACCAACCCACAAACCACAAAACCACCCA
STR2: ACCCAAAACACCAACCAACAACCCAAACCCACCCAACAAACAACCCCAACACCAACCCCACCCACCACAAACCCCACCCAAAAACAAAACCAACACACACACAACCCCCACACCCCCCACCAACCAACCAAACAACCACCCCAAACCAAACCACCAAACCAACCCCCAAACACCAACCCAAAACAAAAAAACCCACAACCACCCCCAACCCCACCACAAAAACCAACCACAAAACCCCCAAACCACCACACCCCACACAACCCCACAACCACAAAAACACACCAAACAACAAAAAACACACAAAAACACCCAACAACAAAAACACCAACACCCCAAAACAAAACCCCAACCCCCCACACACAACCACCCCCAACACACAAAAACCAAACCAACACCAAACACAACACAAACACCCCAACCCCAACCACCACCACCACAAAACCCCAA

### Testing for string length 1188 from the following code block (t=4) alphabets A, C

In [20]:
z = 1188
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.05)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  1188
STR1: AACACAACCACACAAACAAACACACCCCAAACCACACCCAAAAAAAAACCCCACCACCAAAAACCACCACAACAACCCCCAACAACACAAACCACAACAAACACACACCCACAACAAAACCACACCCCACCCAACCCAACAAAAAACCACACCACCACACCCCAAACAAAAACACACAAAACAAACCCACCACCCACCAAACCCACAACAACACCAAAAACCACACACCCCCACACACACCCCAACACACCCCCCACCCCACCAAAAAACCCCCCCCAACAAAACAAACACCAACAAAAAAACACCACACCCACCACCCCCCCCCCCCAAACCAAACAACCACCCCAACAAAAAAACACACAACAAACCCAACCAACACAACAAACACCCCCCACACACCCCAAAACCACCCCCCCCCCACCACCAAACACAAAACCACCCAAACAACAACAACAAAACACCCACCAACAACAAACCCACAACAAACCCAAACCACCAACCAACCCACACCCCAACCAAAACAAAAAACCAAAACCCACCCCCAACCCACCCCACACCAAACCACAAACAAAACAACCAACAACAAAAACCAACACCAACCAACCCAACCAACCCCACACACCAACCAACAAAACACACAAAACCAAAAAAACAAACACCCACCCACACACCAAACCAACAAACCCCCAAACAAAACAACCCCCCAACCACCAACCAACACAACCCACAACCAAAAAACACACCCCAACCAACAAACCCCAAACACCAACACCCCCACCAAAAACCAAACCAACCACAACAACAACCACCACCCACAAAAAAAAAAACCCACCAAAACACCCAAACCAAACCCAACAAAAAAAAACACCCCCACAACCCAACCAAAACACACAAACCAACCCCACCAAAACCACACAAAAAACCACCCCCCCACAAACAACCAAACCCCACCCAACCAAAAACCACCAA

### Testing for string length 2400 from the following code block (t=4) alphabets A, C

In [21]:
z = 2400
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.05)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  2400
STR1: AACAAAACCCCCCCACAAAAAAAACCAACACAACCACCCCCCAACAACCCCAACACCAAACAACCACACCACCACACAAACAACCCAAACCCACCCCACAACCCCAACACACAACACAACACCCCACACCACACAAAACCCACCCCCAACACCCAAACACCACACACACCACCCAACACCCAAAAAAACCCCCCCAACCAACCACACCACACCACCAACCACCAAAAAACCAACCAAAACAACCAACCAAACAACACCACAACACCCCACAACACCCACCAACACAAACCCAAACCCCAACCAACACCCAACCACCAACACCAAAACCACCACCCCAAAAAACACCCAAACCCCAAACCAAAAACCAAACACCACCACACCACCCCAACACACCACCCACCACACCACCCCACCAAAACAACACAACAACCCAAACAAAACCAAACAACACAAACAAACACCCAACACCAACACCCCAAAAACCAAAACACACAACCCCCCCCACAAACAAAAAAACAAACCACACACACCACACAAACAACACCCCCACCAAAAACCCCAACCACCAACACAACCCCCAAACCACCCCAACCCACCAACAACACACACCCCACCAAAACACAACAAAAACCCCACCCACACCACACAAACAAAACCCAAAACACACAACCACCCCAACCACCACACCACAACACAAAAAAACCACCACCAAACCCCACAAACCCAAACAACAAACACACACCCCAACCCCCCAAAACCCACCACCACAACCCACACAACCCCACCCCCAACACCACACACCAACAACCACCCAAACAACAAACAACAACCAAACCCCCACCAAACCACAACAACCAAAAAAAAACCCCCCCAAACACCAAAACAAACCAAAAACAACCACACCAAACCAACACAAACCCAAACCACAAACCACCAAACCAAACCCAAACCAACCACAACCCAAAACCC

############################################################################################################################
############################################################################################################################
############################################################################################################################

### Testing for string length 24 from the following code block (t=3) alphabets A, C

In [22]:
t = 3
alphabets = ['A', 'C',]
start_time = time.time()
allBlocks = finalBlocksGetter(alphabets, t)
end_time = time.time()
time_taken_all_blocks = end_time - start_time
print("Block size", t)
print("ALL BLOCKS - PRECOMPUTE TIME", time_taken_all_blocks, "seconds.")

Block size 3
ALL BLOCKS - PRECOMPUTE TIME 3.038381338119507 seconds.


In [23]:
z = 24
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.05)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  24
STR1: ACCCACCACACCCAACCCCAACAA
STR2: ACCCCCAAAACAAACACCCAACAC
DP EDIT SCORE: 6    FR SCORE: 6.0    PARALLEL FR SCORE: 6.0
DP EDIT TIME: 0.0    FR TIME: 0.01295924186706543    PARALLEL FR TIME: 0.461681604385376


### Testing for string length 48 from the following code block (t=3) alphabets A, C

In [24]:
z = 48
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.05)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  48
STR1: AACCAACCCACCCAAACAAACACACCAACCCCACAAACCACCCCAAAA
STR2: CACACACCCCAACCAACCACCCCCACAAAACAAACAACACACACAACC
DP EDIT SCORE: 18    FR SCORE: 18.0    PARALLEL FR SCORE: 18.0
DP EDIT TIME: 0.004113435745239258    FR TIME: 0.018053770065307617    PARALLEL FR TIME: 0.9098646640777588


### Testing for string length 132 from the following code block (t=3) alphabets A, C

In [25]:
z = 132
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  132
STR1: ACCAACACACCAACCACACCCCACCACCACACACCACAAACCAACAAAACACCAAAACAAACACAAACCAAACCCACCCACCAAACACCCACCCACCCAAACAACCACACCAAACACCCCACCCAACCACAA
STR2: AACCACAACACCACACCAAACACAAAACACAACCCCACCCCACCCAACAAAACAACCACACACACACCCCACCAAAAAACACCCCAAACCCCCAAACACAAACCACCCCCACACAAAAAACACCCCACCCCA
DP EDIT SCORE: 43    FR SCORE: 43.0    PARALLEL FR SCORE: 43.0
DP EDIT TIME: 0.03368830680847168    FR TIME: 0.09506011009216309    PARALLEL FR TIME: 0.5439987182617188


### Testing for string length 516 from the following code block (t=3) alphabets A, C

In [26]:
z = 516
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.005)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  516
STR1: ACCCACCCCCCACCCACACCCCCAAACCCAAAACACACACAAAACCCACCACCACCCCCCCCCAACCACCCCCAACACACCCACCCCCCAAAACCACACCAAAACCACAAAACAACACACCCAACACCAACCAACCAAAACACAACCCACACAAAAAAACACCACAACCAAACACCCACAAAAACAAACCCCACAACACCAAAACCACACAAAACCCCCACAACACAAACCCAAACAAACCAACCCCACCCCCAAAAACCCCAACCAAAAAACACACCAACAAACAACACCCCCCCAACACACACCAAACACCCCAAAAAAACCAACCACCCAAACAACCACAAACCACCCCCCCACCACAAACCAAAACCCAACCAACAAAAACCACACAAACCACCACACCACAACCCCACACAACCCACAAACACCACCCCAAAAAACCAACACACCACCACAAAAAACAAACACCCACAACCACACCCCACCCAACAACCCAAACCACCAACCCAAACCAAC
STR2: CACCCAACCCACCCACCCCCCCCCACCCCCAACCAACCACAACCAACCAACACAAACACCCCACCCCACAACCCCAACACCCAAAAAACAACACAACCCCCCCCCCCACAACCACCCCAACCCCCCCAAACACCACCCACCAAACAACACCACACCACCACCAAAAACCACACCCACAACAAAAACCCCCCAAAAAACCCACAACACCACCCCACACCCCACACCCACAACAACAACACAACACCACCACCACAACCCACCAACACCCAACACAAACCAACCACACACACCAACCCCAACACAAACCCAAAACAAACACCACACCCAAAACCCCCACACAAAACAACAACACAAACAACACCACAACCCCCAACCACAACAAACCACACCCCAAAACAAAACAACCCCCCCCCAACAAAACAAACACAACACACAACACACACCACA

### Testing for string length 1188 from the following code block (t=3) alphabets A, C

In [29]:
z = 1188
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.05)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  1188
STR1: AACACCCCACCAACAAACCCCACAACCACCACCACCACACAAACCCCACAACCCACACCCCAACACAACCACAACAACAAAAAAACCCAAACAACCCCCCACCCACACCAAACCAAACCCACCACACCCCACAACAAACAACAACCAAAACCCAAACAACACCCCCCCAAACCAAAAACACCAACACCCCCAACAAACACCCCACCCCCCACACCCACCCCAACCAAACCAAACCCACCCACCCACACCCACACCCACCCCCACACACCACCCCACAAACACACCCCCAAAACCAACAACAAACACAACCAAAACCAACCAAAAAAACCCAAACCCCCCACAACACCAACCAACCACAACACCCACACACAACCAACAACCAACACCCCCCAACACAACAAACACCACCCCAAACCCCCAACAAAACCCCAACACACCCCCCCAACACCACAACCAACCCACACACAACACCCAAACCACCACACAACACAACCCACAAAAACACACCCCACACAACCACCACACCCACCAAAAACCCCCCCCAACCCCCCCACCAAACCAAACCCACACAACACCACACCACCACAAAAAAAAACAACAACCACCACAACCACAACCAAACCCAAAACCCCCCCAACACAACACACCCCCCCCAACACCCAACACCCCACAACCCCCCAAACCAACAACCACAAACCAAAACCCAAACACCCAACACCCAACACCAACACCCCAAACCCCACAACCCCACCAACCCCCCAACCAACAAACCCAAACCCACAAACACCCCAAACAAAACCACACAACCACCAACCACCACAAAAACCAAACCCCCCCACCACCCAACACAACACCACACACCCACAAAAAAAACAAACCACAAACCAACACACAACCCCAACAACCCAAAACCCCCACACACCCCACCCACCCCACAAAACCAAACCCAACACAACAACACACCCACAC

### Testing for string length 2400 from the following code block (t=3) alphabets A, C

In [31]:
z = 2400
print("Length of strings: ", z)
str1 = createRandomString(z, alphabets)
str2 = createRandomString(z, alphabets)

start_time = time.time()
score1, alignment = global_align(str2, str1, delta)
end_time = time.time()
time_taken_DP_Edit = end_time - start_time


start_time = time.time()
GroundTruthBlock, score2 = FR(str1, str2, t, allBlocks)
end_time = time.time()
time_taken_FR = end_time - start_time

start_time = time.time()
GroundTruthBlock, score3 = FR_Parallel(str1, str2, t, allBlocks, delay=0.09)
end_time = time.time()
time_taken_FR_Parallel = end_time - start_time

# print("STR1", str1)
# print("STR2", str2)
# print("SCORE: ", score1, score2, score3)
# print("TIME: ", time_taken_DP_Edit, time_taken_FR, time_taken_FR_Parallel)

print("STR1:", str1)
print("STR2:", str2)
print("DP EDIT SCORE:", score1, "   FR SCORE:", score2, "   PARALLEL FR SCORE:", score3)
print("DP EDIT TIME:", time_taken_DP_Edit, "   FR TIME:", time_taken_FR, "   PARALLEL FR TIME:", time_taken_FR_Parallel)

Length of strings:  2400
STR1: ACCCCCCCCCACCAACACAACAAACAAACCAACCCACCACACAAACCAACCAAACAAAAACCAACCCAACACCAAAAAAACCAACCCACACCACACCACCCACACCCCCACAACACCCCAACACAAACCAAACCCCAAACCCCAAAAAACAAACAACCCCAACACACAACACAAAACCCACCCACACAAACCCCCACAAAACACAACCACAACCCAAAACACCAACCCCCACCCAACCACACACACCACACCCACCAAACCACCAAAAACACAACAACCCAACCAACCAAACAAAACCACACCCCACACACAAACCCAAAACAACACAACCAAACAAACAACCCAAAACAAAACCACACACAAACACCACCACAAACCAAACCCCAACCACCCCCCCACACACAAACCCAACAAAAAACCACACCAAACCCCAAAAAACCAACACACACACCCCCCAACACCAACAACCACACAACCAAAAAAAACCACACCAACCCACCCCCACCACCACAACCAACCAACCCAAAACCACACCAACAAAAAAACCCCACAACCACACAAACACCCAACAAAACAAACAAACCCACAAACACAAAACAAAAACACAAACCCCCACCCCAAACCCCACAAAAACACAACAAAACACCCACCACCCCCACCCAAACACCCACAACCCAAAAAACCAAAAACCAAACAACCCCAAAAACAACCCACCCACCCCCACCCCACACCCCACCACCCAACCACCAACACCACCCACACAAACAAAACCAAAACAACAACCCCCCCACCCAAAACACAACAACACACCCACAACAAAACCCAACAACACACACAAAAACAACAACACCCAAACCCCACACCCCCACAAACCCACCCACACCAAACAACAAACCCCCCCCCACCCACCACAACCCCACAAACCACCCCACCACACCAAACACCACCCAAACACCACCCCAAACCAAA